# Jaeger-query with trace DSL

This notebook loads a trace from Jaeger query service via gRPC and runs analysis on it.

## Install library to the local maven repository
This step is only needed if trace DSL source code has been modified.
Open terminal in Jupyter and run the following command:
```
cd work && ./mvnw clean install -DskipTests
```

In [1]:
%%loadFromPOM
<dependency>
  <groupId>com.google.protobuf</groupId>
  <artifactId>protobuf-java</artifactId>
  <version>3.11.4</version>
</dependency>
<dependency>
  <groupId>com.google.protobuf</groupId>
  <artifactId>protobuf-java-util</artifactId>
  <version>3.11.4</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-protobuf</artifactId>
  <version>1.28.0</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-netty</artifactId>
  <version>1.28.0</version>
</dependency>
<dependency>
  <groupId>io.grpc</groupId>
  <artifactId>grpc-stub</artifactId>
  <version>1.28.0</version>
</dependency>

<dependency>
    <groupId>io.prometheus</groupId>
    <artifactId>simpleclient_httpserver</artifactId>
    <version>0.7.0</version>
</dependency>
<dependency>
    <groupId>org.apache.tinkerpop</groupId>
    <artifactId>tinkergraph-gremlin</artifactId>
    <version>3.4.3</version>
</dependency>
<dependency>
    <groupId>log4j</groupId>
    <artifactId>log4j</artifactId>
    <version>1.2.17</version>
</dependency>


In [2]:
%maven io.jaegertracing:jaeger-tracedsl:0.2.1-SNAPSHOT
%maven io.jaegertracing:jaeger-proto:0.2.1-SNAPSHOT

System.out.println(io.jaegertracing.analytics.gremlin.Keys.class);

class io.jaegertracing.analytics.gremlin.Keys


## Run analysis
It calculates maximum trace height and network latencies. It connects to Jaeger query service via gRPC.

In [56]:
import org.apache.tinkerpop.gremlin.structure.Graph;
import org.apache.tinkerpop.gremlin.structure.Vertex;
import io.jaegertracing.analytics.gremlin.*;
import io.jaegertracing.analytics.model.*;
import io.jaegertracing.analytics.*;
import io.jaegertracing.analytics.NetworkLatency.*;
import io.grpc.ManagedChannel;
import io.grpc.ManagedChannelBuilder;
import io.jaegertracing.api_v2.Query.*;
import java.util.*;
import com.google.protobuf.ByteString;
import io.jaegertracing.api_v2.QueryServiceGrpc.QueryServiceBlockingStub;
import io.jaegertracing.api_v2.QueryServiceGrpc;

String queryHostPort = "jaeger:16686";
String traceIdStr = "2cb1f0c274c3b4a5";

ManagedChannel channel = ManagedChannelBuilder.forTarget(queryHostPort).usePlaintext().build();
QueryServiceBlockingStub queryService = QueryServiceGrpc.newBlockingStub(channel);

ByteString traceId = Converter.toProtoId(traceIdStr);
Iterator<SpansResponseChunk> traceProto = queryService.getTrace(GetTraceRequest.newBuilder().setTraceId(traceId).build());

Trace trace = Converter.toModel(traceProto.next().getSpansList());
Graph graph = GraphCreator.create(trace);

int height = TraceHeight.calculate(graph);
Map<Name, Set<Double>> networkLatencies = NetworkLatency.calculate(graph);

System.out.printf("Trace height = %d\n", height);
System.out.printf("Network latencies = %s\n", networkLatencies);
System.out.println()

Trace height = 3
Network latencies = {Name{client='frontend', server='customer'}=[0.001021]}



## Write your own data analysis with Trace DSL and Apache Gremlin

In [55]:
// Are two spans connected?
TraceTraversal<Vertex, Vertex> traversal = graph.traversal(TraceTraversalSource.class)
        .hasName("HTTP GET: /customer")
        .repeat(__.out())
        .until(__.hasName("SQL SELECT"));

traversal.forEachRemaining(v -> Util.printVertex(v));


SQL SELECT[2cb1f0c274c3b4a5:219d2ad95fda3bdf], tags = [traceId, request, operationName, serviceName, parentId, spanId, duration, internal.span.format, span.kind, peer.service, sql.query, startTime, span]
